In [82]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [83]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

In [84]:
education_df = pd.read_sql_query('select * from useducation',con=engine)

# No need for an open connection, 
# because you're only doing a single query
engine.dispose()

In [85]:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

In [86]:
# 1. Determine all the variable types and find the fraction of the missing values for each variable.

education_df.isnull().sum()*100/education_df.isnull().count()

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                          17.627346
TOTAL_REVENUE                   14.209115
FEDERAL_REVENUE                 14.209115
STATE_REVENUE                   14.209115
LOCAL_REVENUE                   14.209115
TOTAL_EXPENDITURE               14.209115
INSTRUCTION_EXPENDITURE         14.209115
SUPPORT_SERVICES_EXPENDITURE    14.209115
OTHER_EXPENDITURE               17.627346
CAPITAL_OUTLAY_EXPENDITURE      14.209115
GRADES_PK_G                     11.595174
GRADES_KG_G                      8.847185
GRADES_4_G                       8.780161
GRADES_8_G                       8.780161
GRADES_12_G                      8.780161
GRADES_1_8_G                     8.780161
GRADES_9_12_G                    8.780161
GRADES_ALL_G                    11.595174
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

In [46]:
# 2. Notice that the data has a time dimension (year). For this assessment, 
# forget about time and treat all the observations as if they're from the
# same year. Choose a strategy to deal with the missing values for each 
# variable. For which variables would filling in the missing values with 
# some value make sense? For which might tossing out the records entirely 
# make sense?

# Avg scores are all missing 60+ percent of data and should not be used 
# in their current state for any analysis.

# Remaining missing values are under 20% so will be filled in using the mean
# of the corresponding column.

education_df["ENROLL"].fillna(education_df["ENROLL"].mean(), inplace = True)
education_df["TOTAL_REVENUE"].fillna(education_df["TOTAL_REVENUE"].mean(), inplace = True)
education_df["FEDERAL_REVENUE"].fillna(education_df["FEDERAL_REVENUE"].mean(), inplace = True)
education_df["STATE_REVENUE"].fillna(education_df["STATE_REVENUE"].mean(), inplace = True)
education_df["LOCAL_REVENUE"].fillna(education_df["LOCAL_REVENUE"].mean(), inplace = True)
education_df["TOTAL_EXPENDITURE"].fillna(education_df["TOTAL_EXPENDITURE"].mean(), inplace = True)
education_df["INSTRUCTION_EXPENDITURE"].fillna(education_df["INSTRUCTION_EXPENDITURE"].mean(), inplace = True)
education_df["SUPPORT_SERVICES_EXPENDITURE"].fillna(education_df["SUPPORT_SERVICES_EXPENDITURE"].mean(), inplace = True)
education_df["OTHER_EXPENDITURE"].fillna(education_df["OTHER_EXPENDITURE"].mean(), inplace = True)
education_df["CAPITAL_OUTLAY_EXPENDITURE"].fillna(education_df["CAPITAL_OUTLAY_EXPENDITURE"].mean(), inplace = True)
education_df["GRADES_PK_G"].fillna(education_df["GRADES_PK_G"].mean(), inplace = True)
education_df["GRADES_KG_G"].fillna(education_df["GRADES_KG_G"].mean(), inplace = True)
education_df["GRADES_4_G"].fillna(education_df["GRADES_4_G"].mean(), inplace = True)
education_df["GRADES_8_G"].fillna(education_df["GRADES_8_G"].mean(), inplace = True)
education_df["GRADES_12_G"].fillna(education_df["GRADES_12_G"].mean(), inplace = True)
education_df["GRADES_1_8_G"].fillna(education_df["GRADES_1_8_G"].mean(), inplace = True)
education_df["GRADES_9_12_G"].fillna(education_df["GRADES_9_12_G"].mean(), inplace = True)
education_df["GRADES_ALL_G"].fillna(education_df["GRADES_ALL_G"].mean(), inplace = True)

education_df.isnull().sum()*100/education_df.isnull().count()

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                           0.000000
TOTAL_REVENUE                    0.000000
FEDERAL_REVENUE                  0.000000
STATE_REVENUE                    0.000000
LOCAL_REVENUE                    0.000000
TOTAL_EXPENDITURE                0.000000
INSTRUCTION_EXPENDITURE          0.000000
SUPPORT_SERVICES_EXPENDITURE     0.000000
OTHER_EXPENDITURE                0.000000
CAPITAL_OUTLAY_EXPENDITURE       0.000000
GRADES_PK_G                      0.000000
GRADES_KG_G                      0.000000
GRADES_4_G                       0.000000
GRADES_8_G                       0.000000
GRADES_12_G                      0.000000
GRADES_1_8_G                     0.000000
GRADES_9_12_G                    0.000000
GRADES_ALL_G                     0.000000
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

In [73]:
# 3. Now, take into account the time factor. Replicate your second answer, 
# but this time, fill in the missing values by using a statistic that is 
# calculated within the year of the observation. For example, if you want 
# to fill in a missing value for a variable with the mean of that variable, 
# calculate the mean by using only the observations for that specific year.

for col in education_df:
    if (education_df[col].isnull().sum()*100/education_df[col].isnull().count() > 0.0 and
        education_df[col].isnull().sum()*100/education_df[col].isnull().count() < 20.0):
        for year in education_df['YEAR'].unique():
            year_mean = education_df[education_df['YEAR'] == year][col].mean()
            education_df.loc[education_df['YEAR'] == year, col] = education_df.loc[education_df['YEAR'] == year, col].fillna(year_mean)
    
education_df.isnull().sum()*100/education_df.isnull().count()          

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                           7.171582
TOTAL_REVENUE                    3.418231
FEDERAL_REVENUE                  3.418231
STATE_REVENUE                    3.418231
LOCAL_REVENUE                    3.418231
TOTAL_EXPENDITURE                3.418231
INSTRUCTION_EXPENDITURE          3.418231
SUPPORT_SERVICES_EXPENDITURE     3.418231
OTHER_EXPENDITURE                7.171582
CAPITAL_OUTLAY_EXPENDITURE       3.418231
GRADES_PK_G                      6.836461
GRADES_KG_G                      6.836461
GRADES_4_G                       6.836461
GRADES_8_G                       6.836461
GRADES_12_G                      6.836461
GRADES_1_8_G                     6.836461
GRADES_9_12_G                    6.836461
GRADES_ALL_G                     6.836461
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

In [87]:
# 4. This time, fill in the missing values using interpolation (extrapolation).

education_df = education_df.interpolate(method = 'linear', limit_area = 'outside')

education_df.isnull().sum()*100/education_df.isnull().count() 

PRIMARY_KEY                      0.000000
STATE                            0.000000
YEAR                             0.000000
ENROLL                           3.820375
TOTAL_REVENUE                    0.402145
FEDERAL_REVENUE                  0.402145
STATE_REVENUE                    0.402145
LOCAL_REVENUE                    0.402145
TOTAL_EXPENDITURE                0.402145
INSTRUCTION_EXPENDITURE          0.402145
SUPPORT_SERVICES_EXPENDITURE     0.402145
OTHER_EXPENDITURE                3.820375
CAPITAL_OUTLAY_EXPENDITURE       0.402145
GRADES_PK_G                      8.512064
GRADES_KG_G                      5.831099
GRADES_4_G                       5.764075
GRADES_8_G                       5.764075
GRADES_12_G                      5.764075
GRADES_1_8_G                     5.764075
GRADES_9_12_G                    5.764075
GRADES_ALL_G                     8.512064
AVG_MATH_4_SCORE                64.075067
AVG_MATH_8_SCORE                64.343164
AVG_READING_4_SCORE             64

In [10]:
# 5. Compare your results for the second, third, and fourth questions. Do you 
# find any meaningful differences?

# Manually replacing missing values with our own choosing obviously allows us
# to remove all missing values, where as the other two methods cannot guarantee
# that result. Replacing values with a qualifier that requires another valid 
# value such as year means if we dont have data within that givin year then the
# missing values wont be filled. Interpolating also requires valid data to present
# wherever we are pulling data from to do said interpolating. With that said
# interpolating at just a basic level filled in the majority of missing values 
# in most columns.